# BrainGraphNet - Data Exploration
## Explore synthetic brain connectivity data and graphs

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.synthetic_generator import SyntheticBrainDataGenerator
from src.data.graph_builder import GraphBuilder
from src.utils.config_parser import load_config
from src.visualization.connectivity_plots import plot_connectivity_matrix, plot_temporal_connectivity_evolution
from src.visualization.brain_plots import plot_brain_network

# Set style
sns.set_style('whitegrid')
%matplotlib inline

## 1. Load Configuration

In [ ]:
config = load_config('../config.yaml')
print("Configuration loaded successfully!")
print(f"Data source: {config['data']['source']}")
print(f"Number of regions: {config['data']['atlas']['num_regions']}")

## 2. Generate Synthetic Data

In [ ]:
# Generate data
generator = SyntheticBrainDataGenerator(config)
connectivity_data, labels = generator.generate()

print(f"Generated connectivity data shape: {connectivity_data.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Number of subjects: {connectivity_data.shape[0]}")
print(f"Number of timepoints: {connectivity_data.shape[1]}")
print(f"Number of regions: {connectivity_data.shape[2]}")

## 3. Visualize Connectivity Matrices

In [ ]:
# Plot connectivity matrix for first subject, first timepoint
subject_id = 0
timepoint = 0

connectivity = connectivity_data[subject_id, timepoint]
plot_connectivity_matrix(
    connectivity,
    title=f"Subject {subject_id}, Timepoint {timepoint}"
)

## 4. Temporal Evolution

In [ ]:
# Plot temporal evolution for one subject
subject_sequence = connectivity_data[subject_id]
plot_temporal_connectivity_evolution(subject_sequence)

## 5. Connectivity Statistics

In [ ]:
# Compute statistics
print("Connectivity Statistics:")
print(f"Mean: {connectivity.mean():.3f}")
print(f"Std: {connectivity.std():.3f}")
print(f"Min: {connectivity.min():.3f}")
print(f"Max: {connectivity.max():.3f}")

# Distribution
triu_indices = np.triu_indices_from(connectivity, k=1)
values = connectivity[triu_indices]

plt.figure(figsize=(10, 6))
plt.hist(values, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Connectivity Strength')
plt.ylabel('Frequency')
plt.title('Distribution of Connectivity Values')
plt.grid(alpha=0.3)
plt.show()

## 6. Build Graphs

In [ ]:
# Build graphs from connectivity matrices
graph_builder = GraphBuilder(config)
graphs = graph_builder.build_temporal_graphs(subject_sequence)

print(f"Number of graphs: {len(graphs)}")
print(f"First graph: {graphs[0]}")
print(f"Number of nodes: {graphs[0].num_nodes}")
print(f"Number of edges: {graphs[0].edge_index.shape[1]}")

## 7. Visualize Brain Network

In [ ]:
# Plot brain network
plot_brain_network(connectivity, threshold=0.3)

## 8. Compare Healthy vs Disease

In [ ]:
# Compare healthy and disease subjects
healthy_idx = np.where(labels == 0)[0][0]
disease_idx = np.where(labels == 1)[0][0]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Healthy
im1 = axes[0].imshow(connectivity_data[healthy_idx, -1], cmap='RdBu_r', vmin=-1, vmax=1)
axes[0].set_title('Healthy Subject (Final Timepoint)', fontweight='bold')
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# Disease
im2 = axes[1].imshow(connectivity_data[disease_idx, -1], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1].set_title('Disease Subject (Final Timepoint)', fontweight='bold')
plt.colorbar(im2, ax=axes[1], fraction=0.046)

plt.tight_layout()
plt.show()

## 9. Graph Statistics Over Time

In [ ]:
# Compute graph statistics over time
num_edges = [g.edge_index.shape[1] for g in graphs]
avg_degree = [g.edge_index.shape[1] / g.num_nodes for g in graphs]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(range(1, len(graphs)+1), num_edges, marker='o', linewidth=2)
axes[0].set_xlabel('Timepoint')
axes[0].set_ylabel('Number of Edges')
axes[0].set_title('Edge Count Evolution')
axes[0].grid(alpha=0.3)

axes[1].plot(range(1, len(graphs)+1), avg_degree, marker='s', linewidth=2, color='orange')
axes[1].set_xlabel('Timepoint')
axes[1].set_ylabel('Average Degree')
axes[1].set_title('Average Degree Evolution')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Loading configuration
- Generating synthetic brain connectivity data
- Visualizing connectivity matrices
- Analyzing temporal evolution
- Building graph representations
- Computing graph statistics

Next: Train temporal GNN model in `02_model_analysis.ipynb`